## Build Model

### Using the python API.

Here we are going to build a model using the HydroMT-FIAT python API.

In [ ]:
# Imports
import tempfile
from pathlib import Path

from hydromt._utils import log

from hydromt_fiat import FIATModel

Lets make sure that there is a directory to build to.

In [ ]:
# Use the built-in tempfile package
temp_dir = tempfile.TemporaryDirectory(delete=False)
p = Path(temp_dir.name)

Now lets setup the Model.

In [ ]:
# Create a model in write mode.
log._setuplog()
model = FIATModel(root=p, mode="w+")

Next step is to update the configurations and add a region for the model to know to collect data in.

Simply done.